In [2]:
import numpy as np
import pandas as pd
from mne.io import RawArray
from mne.channels import read_montage
from mne.epochs import concatenate_epochs
from mne import create_info, find_events, Epochs, concatenate_raws, pick_types
from mne.decoding import CSP

from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.lda import LDA
from sklearn.metrics import roc_auc_score
from glob import glob

from scipy.signal import butter, lfilter, convolve, boxcar

In [3]:
def creat_mne_raw_object(fname,read_events=True):
    """Create a mne raw instance from csv file"""
    # Read EEG file
    data = pd.read_csv(fname)
    
    # get chanel names
    ch_names = list(data.columns[1:])
    
    # read EEG standard montage from mne
    montage = read_montage('standard_1005',ch_names)

    ch_type = ['eeg']*len(ch_names)
    data = 1e-6*np.array(data[ch_names]).T
    
    if read_events:
        # events file
        ev_fname = fname.replace('_data','_events')
        # read event file
        events = pd.read_csv(ev_fname)
        events_names = events.columns[1:]
        events_data = np.array(events[events_names]).T
        
        # define channel type, the first is EEG, the last 6 are stimulations
        ch_type.extend(['stim']*6)
        ch_names.extend(events_names)
        # concatenate event file and data
        data = np.concatenate((data,events_data))
        
    # create and populate MNE info structure
    info = create_info(ch_names,sfreq=500.0, ch_types=ch_type, montage=montage)
    info['filename'] = fname
    
    # create raw object 
    raw = RawArray(data,info,verbose=False)
    
    return raw

In [4]:
subjects = range(1,13)
ids_tot = []

# Array for all subjects train & labels
feattr_tot = []
labels_tot = []

# Array for all subjects test & labels
featte_tot = []
y_tot = []

# design a butterworth bandpass filter 
freqs = [7, 30]
b,a = butter(5,np.array(freqs)/250.0,btype='bandpass')

# CSP parameters
# Number of spatial filter to use
nfilters = 4

# convolution
# window for smoothing features
nwin = 250

# training subsample
subsample = 1000

# submission file
submission_file = 'beat_the_benchmark.csv'
cols = ['HandStart','FirstDigitTouch',
        'BothStartLoadPhase','LiftOff',
        'Replace','BothReleased']

In [5]:
for subject in subjects:
    epochs_tot = []
    y = []
    
    ################ READ DATA (FIRST SIX SERIES AS TRAINING DATA) ###########
    fnames = []
    for i in range(1, 7):
        fnames.append('train/subj%d_series%d_data.csv' % (1, i))
    
    # read and concatenate all the files
    raw = concatenate_raws([creat_mne_raw_object(fname) for fname in fnames])
       
    # pick eeg signal
    picks = pick_types(raw.info,eeg=True)
    
    # Filter data for alpha frequency and beta band
    # Note that MNE implement a zero phase (filtfilt) filtering not compatible
    # with the rule of future data.
    # Here we use left filter compatible with this constraint
    raw._data[picks] = lfilter(b,a,raw._data[picks])
    
    ################ CSP Filters training #####################################
    # get event posision corresponding to Replace
    events = find_events(raw,stim_channel='Replace', verbose=False)
    # epochs signal for 1.5 second before the movement
    epochs = Epochs(raw, events, {'during' : 1}, -2, -0.5, proj=False,
                    picks=picks, baseline=None, preload=True,
                    add_eeg_ref=False, verbose=False)
    
    epochs_tot.append(epochs)
    y.extend([1]*len(epochs))
    
    # epochs signal for 1.5 second after the movement, this correspond to the 
    # rest period.
    epochs_rest = Epochs(raw, events, {'after' : 1}, 0.5, 2, proj=False,
                    picks=picks, baseline=None, preload=True,
                    add_eeg_ref=False, verbose=False)
    
    # Workaround to be able to concatenate epochs with MNE
    epochs_rest.times = epochs.times
    
    y.extend([-1]*len(epochs_rest))
    epochs_tot.append(epochs_rest)
        
    # Concatenate all epochs
    epochs = concatenate_epochs(epochs_tot)
    
    # get data 
    X = epochs.get_data()
    y = np.array(y)
    
    # train CSP
    csp = CSP(n_components=nfilters, reg='lws')
    csp.fit(X,y)
    
    ################ Create Training Features #################################x`
    # apply csp filters and rectify signal
    feat = np.dot(csp.filters_[0:nfilters],raw._data[picks])**2
    
    # smoothing by convolution with a rectangle window    
    feattr = np.empty(feat.shape)
    for i in range(nfilters):
        feattr[i] = np.log(convolve(feat[i],boxcar(nwin),'full'))[0:feat.shape[1]]
    
    # training labels
    # they are stored in the 6 last channels of the MNE raw object
    labels = raw._data[32:]
    
    # append to all subject array
    feattr_tot.append(feattr)
    labels_tot.append(labels)
    
    ################ Create test Features #####################################
    # read test data (remaining series of train data)
    fnames = []
    for i in range(7, 9):
        fnames.append('train/subj%d_series%d_data.csv' % (1, i))
        
    raw = concatenate_raws([creat_mne_raw_object(fname) for fname in fnames])
    raw._data[picks] = lfilter(b,a,raw._data[picks])
    
    # get actual y values
    y = raw._data[32:]
    y_tot.append(y.T)
    
    # read ids
    ids = np.concatenate([np.array(pd.read_csv(fname)['id']) for fname in fnames])
    ids_tot.append(ids)
    
    # apply preprocessing on test data
    feat = np.dot(csp.filters_[0:nfilters],raw._data[picks])**2
    featte = np.empty(feat.shape)
    for i in range(nfilters):
        featte[i] = np.log(convolve(feat[i],boxcar(nwin),'full'))[0:feat.shape[1]]
    
    featte_tot.append(featte)

In [6]:
################ Train classifiers ########################################

In [7]:
### Logistic regression ###
pred_tot = []
for subject in range(len(subjects)):
    clf = LogisticRegression()
    pred = np.empty((len(ids),6))
    for i in range(6):
        print('Train subject %d, class %s' % (subject+1, cols[i]))
        clf.fit(feattr_tot[subject][:,::subsample].T, labels_tot[subject][i,::subsample])
        pred[:,i] = clf.predict_proba(featte_tot[subject].T)[:,1]

    pred_tot.append(pred)

Train subject 1, class HandStart
Train subject 1, class FirstDigitTouch
Train subject 1, class BothStartLoadPhase
Train subject 1, class LiftOff
Train subject 1, class Replace
Train subject 1, class BothReleased
Train subject 2, class HandStart
Train subject 2, class FirstDigitTouch
Train subject 2, class BothStartLoadPhase
Train subject 2, class LiftOff
Train subject 2, class Replace
Train subject 2, class BothReleased
Train subject 3, class HandStart
Train subject 3, class FirstDigitTouch
Train subject 3, class BothStartLoadPhase
Train subject 3, class LiftOff
Train subject 3, class Replace
Train subject 3, class BothReleased
Train subject 4, class HandStart
Train subject 4, class FirstDigitTouch
Train subject 4, class BothStartLoadPhase
Train subject 4, class LiftOff
Train subject 4, class Replace
Train subject 4, class BothReleased
Train subject 5, class HandStart
Train subject 5, class FirstDigitTouch
Train subject 5, class BothStartLoadPhase
Train subject 5, class LiftOff
Train s

In [8]:
y_score = np.concatenate(pred_tot)
y_true = np.concatenate(y_tot)

# Convert y_true to int matrix, auc calc will not work with floats
y_true = y_true.astype(int)
auc = roc_auc_score(y_true, y_score, average='macro')
print 'Final Logistic Regression AUC:', auc

Final Logistic Regression AUC: 0.581461941354


In [9]:
### Random forest classifier ###
pred_tot = []
for subject in range(len(subjects)):
    clf = RandomForestClassifier()
    pred = np.empty((len(ids),6))
    for i in range(6):
        print('Train subject %d, class %s' % (subject+1, cols[i]))
        clf.fit(feattr_tot[subject][:,::subsample].T, labels_tot[subject][i,::subsample])
        pred[:,i] = clf.predict_proba(featte_tot[subject].T)[:,1]

    pred_tot.append(pred)

Train subject 1, class HandStart
Train subject 1, class FirstDigitTouch
Train subject 1, class BothStartLoadPhase
Train subject 1, class LiftOff
Train subject 1, class Replace
Train subject 1, class BothReleased
Train subject 2, class HandStart
Train subject 2, class FirstDigitTouch
Train subject 2, class BothStartLoadPhase
Train subject 2, class LiftOff
Train subject 2, class Replace
Train subject 2, class BothReleased
Train subject 3, class HandStart
Train subject 3, class FirstDigitTouch
Train subject 3, class BothStartLoadPhase
Train subject 3, class LiftOff
Train subject 3, class Replace
Train subject 3, class BothReleased
Train subject 4, class HandStart
Train subject 4, class FirstDigitTouch
Train subject 4, class BothStartLoadPhase
Train subject 4, class LiftOff
Train subject 4, class Replace
Train subject 4, class BothReleased
Train subject 5, class HandStart
Train subject 5, class FirstDigitTouch
Train subject 5, class BothStartLoadPhase
Train subject 5, class LiftOff
Train s

In [10]:
y_score = np.concatenate(pred_tot)
y_true = np.concatenate(y_tot)

# Convert y_true to int matrix, auc calc will not work with floats
y_true = y_true.astype(int)
auc = roc_auc_score(y_true, y_score, average='macro')
print 'Final Random Forest AUC:', auc

Final Random Forest AUC: 0.567780186734


In [12]:
### Support vector machine classifier ###
pred_tot = []
for subject in range(len(subjects)):
    clf = SVC(probability=True)
    pred = np.empty((len(ids),6))
    for i in range(6):
        print('Train subject %d, class %s' % (subject+1, cols[i]))
        clf.fit(feattr_tot[subject][:,::subsample].T, labels_tot[subject][i,::subsample])
        pred[:,i] = clf.predict_proba(featte_tot[subject].T)[:,1]

    pred_tot.append(pred)

Train subject 1, class HandStart
Train subject 1, class FirstDigitTouch
Train subject 1, class BothStartLoadPhase
Train subject 1, class LiftOff
Train subject 1, class Replace
Train subject 1, class BothReleased
Train subject 2, class HandStart
Train subject 2, class FirstDigitTouch
Train subject 2, class BothStartLoadPhase
Train subject 2, class LiftOff
Train subject 2, class Replace
Train subject 2, class BothReleased
Train subject 3, class HandStart
Train subject 3, class FirstDigitTouch
Train subject 3, class BothStartLoadPhase
Train subject 3, class LiftOff
Train subject 3, class Replace
Train subject 3, class BothReleased
Train subject 4, class HandStart
Train subject 4, class FirstDigitTouch
Train subject 4, class BothStartLoadPhase
Train subject 4, class LiftOff
Train subject 4, class Replace
Train subject 4, class BothReleased
Train subject 5, class HandStart
Train subject 5, class FirstDigitTouch
Train subject 5, class BothStartLoadPhase
Train subject 5, class LiftOff
Train s

In [13]:
y_score = np.concatenate(pred_tot)
y_true = np.concatenate(y_tot)

# Convert y_true to int matrix, auc calc will not work with floats
y_true = y_true.astype(int)
auc = roc_auc_score(y_true, y_score, average='macro')
print 'Final SVM AUC:', auc

Final SVM AUC: 0.576686898202


In [14]:
### Linear discriminant analysis classifier ###
pred_tot = []
for subject in range(len(subjects)):
    clf = LDA()
    pred = np.empty((len(ids),6))
    for i in range(6):
        print('Train subject %d, class %s' % (subject+1, cols[i]))
        clf.fit(feattr_tot[subject][:,::subsample].T, labels_tot[subject][i,::subsample])
        pred[:,i] = clf.predict_proba(featte_tot[subject].T)[:,1]

    pred_tot.append(pred)

Train subject 1, class HandStart
Train subject 1, class FirstDigitTouch
Train subject 1, class BothStartLoadPhase
Train subject 1, class LiftOff
Train subject 1, class Replace
Train subject 1, class BothReleased
Train subject 2, class HandStart
Train subject 2, class FirstDigitTouch
Train subject 2, class BothStartLoadPhase
Train subject 2, class LiftOff
Train subject 2, class Replace
Train subject 2, class BothReleased
Train subject 3, class HandStart
Train subject 3, class FirstDigitTouch
Train subject 3, class BothStartLoadPhase
Train subject 3, class LiftOff
Train subject 3, class Replace
Train subject 3, class BothReleased
Train subject 4, class HandStart
Train subject 4, class FirstDigitTouch
Train subject 4, class BothStartLoadPhase
Train subject 4, class LiftOff
Train subject 4, class Replace
Train subject 4, class BothReleased
Train subject 5, class HandStart
Train subject 5, class FirstDigitTouch
Train subject 5, class BothStartLoadPhase
Train subject 5, class LiftOff
Train s

In [15]:
y_score = np.concatenate(pred_tot)
y_true = np.concatenate(y_tot)

# Convert y_true to int matrix, auc calc will not work with floats
y_true = y_true.astype(int)
auc = roc_auc_score(y_true, y_score, average='macro')
print 'Final LDA AUC:', auc

Final LDA AUC: 0.587721132395


In [16]:
### Gradient boosting classifier ###
pred_tot = []
for subject in range(len(subjects)):
    clf = GradientBoostingClassifier()
    pred = np.empty((len(ids),6))
    for i in range(6):
        print('Train subject %d, class %s' % (subject+1, cols[i]))
        clf.fit(feattr_tot[subject][:,::subsample].T, labels_tot[subject][i,::subsample])
        pred[:,i] = clf.predict_proba(featte_tot[subject].T)[:,1]

    pred_tot.append(pred)

Train subject 1, class HandStart
Train subject 1, class FirstDigitTouch
Train subject 1, class BothStartLoadPhase
Train subject 1, class LiftOff
Train subject 1, class Replace
Train subject 1, class BothReleased
Train subject 2, class HandStart
Train subject 2, class FirstDigitTouch
Train subject 2, class BothStartLoadPhase
Train subject 2, class LiftOff
Train subject 2, class Replace
Train subject 2, class BothReleased
Train subject 3, class HandStart
Train subject 3, class FirstDigitTouch
Train subject 3, class BothStartLoadPhase
Train subject 3, class LiftOff
Train subject 3, class Replace
Train subject 3, class BothReleased
Train subject 4, class HandStart
Train subject 4, class FirstDigitTouch
Train subject 4, class BothStartLoadPhase
Train subject 4, class LiftOff
Train subject 4, class Replace
Train subject 4, class BothReleased
Train subject 5, class HandStart
Train subject 5, class FirstDigitTouch
Train subject 5, class BothStartLoadPhase
Train subject 5, class LiftOff
Train s

In [17]:
y_score = np.concatenate(pred_tot)
y_true = np.concatenate(y_tot)

# Convert y_true to int matrix, auc calc will not work with floats
y_true = y_true.astype(int)
auc = roc_auc_score(y_true, y_score, average='macro')
print 'Final Gradient Boosting AUC:', auc

Final Gradient Boosting AUC: 0.656767059579
